In [1]:
import numpy as np
import pandas as pd


In [2]:
train = pd.read_csv("https://bit.ly/fc-ml-titanic")

In [4]:
# csv 파일의 dataset 가장 앞의 5개를 보여줌 (default 대신 정수를 넣으면 개수만큼 보여줌)
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* passengerId : boarding ID num
* survived : 1=live, 0=death
* Pclass : cabin level
* Name
* Sex : female, male
* SibSp : bro, sis, wife/husband num
* Parch : parents, son/dauther num
* Fare, cabin Num
* Embarked : on board port

### dividing train / vaildation dataset  (전처리)
1. define btween feature and label
2. dividing feature and label

In [5]:
# 학습에 필요한
# X로 사용할 값 (제공)
feature = ["Pclass", "Sex", "Age", "Fare"]
# Y로 사용할 값 (정답, 예측/비교값)
label = ["Survived"]


In [7]:
# X, Y에 대해 train 할 값의 일부를 확인 (제대로 값을 가지고 오는지 확인)
train[feature].head()
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [8]:
# 학습전 결손 데이터가 있는지에 대해서 확인해서 가공해줘야함
# 기존 자료가 왜곡되지 않도록 함
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
# 데이터셋 나누기
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train[feature], train[label], test_size=0.2, random_state=123)

In [13]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(712, 4)
(712, 1)
(179, 4)
(179, 1)
